###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Comprehensive Plan Data Pull  

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']
#api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

## Read In Data Guide

The "head" should never be more than 2 variables, and the "tail" never more than 2. Since we can pull 50 variables at once this means that we can systematically program in 46 variables for each pull, so that's:
+ dg1: ID's 1 - 46  
+ dg2: ID's 47-92  
+ dg3: ID's 93-138  
+ dg4: ID's 139-184  
+ dg5: ID's 185-230  
+ dg6: ID's 231-276  
+ dg7: ID's 277-322  
+ dg8: ID's 323-368  
+ dg9: ID's 369-414  
+ dg10: ID's 415-460  
+ dg11: ID's 461-506  
+ dg12: ID's 507-552  
+ dg13: ID's 553-598  
+ dg14: ID's 599-644  
+ dg15: ID's 645-690  
+ dg16: ID's 691-736  
+ dg17: ID's 737-782  
+ dg18: ID's 783-828 
+ dg19: ID's 829 - 874  
+ dg20: ID's 875 - 920  
+ dg21: ID's 921 - 966  
+ dg22: ID's 967 - 1012  
+ dg23: ID's 1013 - 1058  
+ dg24: ID's 1059 - 1104  
+ dg25: ID's 1105 - 1150  
+ dg26: ID's 1151 - 1196  
+ dg27: ID's 1197 - 1242

**This data guide stops at ID 1217 as of 7/8/2022.**

In [5]:
dataguide = pd.read_csv('../../data guides/DATA GUIDE ACS2020.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
dg9 = dataguide[dataguide['ID'].between(369, 414)]
dg10 = dataguide[dataguide['ID'].between(415, 460)]
dg11 = dataguide[dataguide['ID'].between(461, 506)]
dg12 = dataguide[dataguide['ID'].between(507, 552)]
dg13 = dataguide[dataguide['ID'].between(553, 598)]
dg14 = dataguide[dataguide['ID'].between(599, 644)]
dg15 = dataguide[dataguide['ID'].between(645, 690)]
dg16 = dataguide[dataguide['ID'].between(691, 736)]
dg17 = dataguide[dataguide['ID'].between(737, 782)]
dg18 = dataguide[dataguide['ID'].between(783, 828)]
dg19 = dataguide[dataguide['ID'].between(829, 874)]
dg20 = dataguide[dataguide['ID'].between(875, 920)]
dg21 = dataguide[dataguide['ID'].between(921, 966)]
dg22 = dataguide[dataguide['ID'].between(967, 1012)]
dg23 = dataguide[dataguide['ID'].between(1013, 1058)]
dg24 = dataguide[dataguide['ID'].between(1059, 1104)]
dg25 = dataguide[dataguide['ID'].between(1105, 1150)]
dg26 = dataguide[dataguide['ID'].between(1151, 1196)]
dg27 = dataguide[dataguide['ID'].between(1197, 1242)]

## One Through Twenty Seven

In [7]:
# ONE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg1['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg1['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
onepull = df
print('Okay Finished')

Okay Finished


In [9]:
#placemarker
one = onepull

In [10]:
#TWO
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg2['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg2['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twopull = df
print('Okay Finished')

Okay Finished


In [11]:
two = twopull

In [12]:
# THREE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg3['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg3['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
threepull = df
print('Okay Finished')

Okay Finished


In [13]:
three = threepull

In [14]:
# FOUR
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg4['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg4['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fourpull = df
print('Okay Finished')

Okay Finished


In [15]:
four = fourpull

In [16]:
# FIVE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg5['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg5['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fivepull = df
print('Okay Finished')

Okay Finished


In [17]:
five = fivepull

In [18]:
# SIX
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg6['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg6['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
sixpull = df
print('Okay Finished')

Okay Finished


In [19]:
six = sixpull

In [20]:
# SEVEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg7['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg7['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
sevenpull = df
print('Okay Finished')

Okay Finished


In [21]:
seven = sevenpull

In [22]:
# EIGHT
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg8['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg8['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
eightpull = df
print('Okay Finished')

Okay Finished


In [23]:
eight = eightpull

In [24]:
# NINE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg9['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg9['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
ninepull = df
print('Okay Finished')

Okay Finished


In [25]:
nine = ninepull

In [26]:
# TEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg10['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg10['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
tenpull = df
print('Okay Finished')

Okay Finished


In [27]:
ten = tenpull

In [28]:
# ELEVEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg11['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg11['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
elevenpull = df
print('Okay Finished')

Okay Finished


In [29]:
eleven = elevenpull

In [30]:
# TWELVE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg12['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg12['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twelvepull = df
print('Okay Finished')

Okay Finished


In [31]:
twelve = twelvepull

In [32]:
# THIRTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg13['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg13['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
thirteenpull = df
print('Okay Finished')

Okay Finished


In [33]:
thirteen = thirteenpull

In [34]:
# FOURTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg14['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg14['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fourteenpull = df
print('Okay Finished')

Okay Finished


In [35]:
fourteen = fourteenpull

In [36]:
# FIFTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg15['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg15['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
fifteenpull = df
print('Okay Finished')

Okay Finished


In [37]:
fifteen = fifteenpull

In [38]:
# SIXTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg16['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg16['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
sixteenpull = df
print('Okay Finished')

Okay Finished


In [39]:
sixteen = sixteenpull

In [40]:
# SEVENTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg17['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg17['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
seventeenpull = df
print('Okay Finished')

Okay Finished


In [41]:
seventeen = seventeenpull

In [42]:
# EIGHTEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg18['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg18['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
eighteenpull = df
print('Okay Finished')

Okay Finished


In [43]:
eighteen = eighteenpull

In [44]:
# NINETEEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg19['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg19['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
nineteenpull = df
print('Okay Finished')

Okay Finished


In [45]:
nineteen = nineteenpull

In [46]:
# TWENTY
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg20['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg20['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentypull = df
print('Okay Finished')

Okay Finished


In [47]:
twenty = twentypull

In [48]:
# TWENTYONE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg21['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg21['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentyonepull = df
print('Okay Finished')

Okay Finished


In [49]:
twentyone = twentyonepull

In [50]:
# TWENTYTWO
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg22['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg22['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentytwopull = df
print('Okay Finished')

Okay Finished


In [51]:
twentytwo = twentytwopull

In [52]:
# TWENTYTHREE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg23['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg23['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentythreepull = df
print('Okay Finished')

Okay Finished


In [53]:
twentythree = twentythreepull

In [54]:
# TWENTYFOUR
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg24['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg24['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentyfourpull = df
print('Okay Finished')

Okay Finished


In [55]:
twentyfour = twentyfourpull

In [56]:
# TWENTYFIVE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg25['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg25['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentyfivepull = df
print('Okay Finished')

Okay Finished


In [57]:
twentyfive = twentyfivepull

In [58]:
# TWENTYSIX
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg26['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg26['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentysixpull = df
print('Okay Finished')

Okay Finished


In [59]:
twentysix = twentysixpull

In [60]:
# TWENTYSEVEN
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg27['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg27['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twentysevenpull = df
print('Okay Finished')

Okay Finished


In [61]:
twentyseven = twentysevenpull

In [62]:
last = twentyseven

In [63]:
dfs = [two,three,four,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen,fifteen,sixteen,seventeen,eighteen,nineteen,twenty,twentyone,twentytwo,twentythree,
      twentyfour,twentyfive,twentysix]

In [64]:
for df in dfs:
    df = df.drop(columns = ['NAME','StateFIPS','GeoFIPS'], inplace = True)

In [65]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [66]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [67]:
dfs = [one, df_merged, last]
data = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [68]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

,,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,healthcoverage_total_healthcare_series,healthcoverage_total_mallhealthcare,healthcoverage_mu6_all,healthcoverage_mu6_w,healthcoverage_mu6_wo,healthcoverage_m6to18_all,healthcoverage_m6to18_w,healthcoverage_m6to18_wo,healthcoverage_m19to25_all,healthcoverage_m19to25_w,healthcoverage_m19to25_wo,healthcoverage_m26to34_all,healthcoverage_m26to34_w,healthcoverage_m26to34_wo,healthcoverage_m35to44_all,healthcoverage_m35to44_w,healthcoverage_m35to44_wo,healthcoverage_m45to54_all,healthcoverage_m45to54_w,healthcoverage_m45to54_wo,healthcoverage_m55to64_all,healthcoverage_m55to64_w,healthcoverage_m55to64_wo,healthcoverage_m65to74_all,healthcoverage_m65to74_w,healthcoverage_m65to74_wo,healthcoverage_m75+_all,healthcoverage_m75+_w,healthcoverage_m75+_wo,healthcoverage_total_fallhealthcare,healthcoverage_fu6_all,healthcoverage_fu6_w,healthcoverage_fu6_wo,healthcoverage_f6to18_all,healthcoverage_f6to18_w,healthcoverage_f6to18_wo,healthcoverage_f19to25_all,healthcoverage_f19to25_w,healthcoverage_f19to25_wo,healthcoverage_f26to34_all,healthcoverage_f26to34_w,healthcoverage_f26to34_wo,healthcoverage_f35to44_all,healthcoverage_f35to44_w,healthcoverage_f35to44_wo,healthcoverage_f45to54_all,healthcoverage_f45to54_w,healthcoverage_f45to54_wo,healthcoverage_f55to64_all,healthcoverage_f55to64_w,healthcoverage_f55to64_wo,healthcoverage_f65to74_all,healthcoverage_f65to74_w,healthcoverage_f65to74_wo,healthcoverage_f75+_all,healthcoverage_f75+_w,healthcoverage_f75+_wo,healthcoverage_total_privatehealthcare_series,healthcoverage_total_mprivate,healthcoverage_mu6_privateall,healthcoverage_mu6_wprivate,healthcoverage_mu6_woprivate,healthcoverage_m6to18_privateall,healthcoverage_m6to18_wprivate,healthcoverage_m6to18_woprivate,healthcoverage_m19to25_privateall,healthcoverage_m19to25_wprivate,healthcoverage_m19to25_woprivate,healthcoverage_m26to34_privateall,healthcoverage_m26to34_wprivate,healthcoverage_m26to34_woprivate,healthcoverage_m35to44_privateall,healthcoverage_m35to44_wprivate,healthcoverage_m35to44_woprivate,healthcoverage_m45to54_privateall,healthcoverage_m45to54_wprivate,healthcoverage_m45to54_woprivate,healthcoverage_m55to64_privateall,healthcoverage_m55to64_wprivate,healthcoverage_m55to64_woprivate,healthcoverage_m65to74_privateall,healthcoverage_m65to74_wprivate,healthcoverage_m65to74_woprivate,healthcoverage_m75+_privateall,healthcoverage_m75+_wprivate,healthcoverage_m75+_woprivate,healthcoverage_total_fprivate,healthcoverage_fu6_privateall,healthcoverage_fu6_wprivate,healthcoverage_fu6_woprivate,healthcoverage_f6to18_privateall,healthcoverage_f6to18_wprivate,healthcoverage_f6to18_woprivate,healthcoverage_f19to25_privateall,healthcoverage_f19to25_wprivate,healthcoverage_f19to25_woprivate,healthcoverage_f26to34_privateall,healthcoverage_f26to34_wprivate,healthcoverage_f26to34_woprivate,healthcoverage_f35to44_privateall,healthcoverage_f35to44_wprivate,healthcoverage_f35to44_woprivate,healthcoverage_f45to54_privateall,healthcoverage_f45to54_wprivate,healthcoverage_f45to54_woprivate,healthcoverage_f55to64_privateall,healthcoverage_f55to64_wprivat

In [69]:
transp = data.transpose()
transp.head()

NAME,"Montgomery County, Tennessee","Rutherford County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Cheatham County, Tennessee","Macon County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Scottsville city, Kentucky","Franklin city, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee",Tennessee,United States
GEO_ID,0500000US47125,0500000US47149,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US47021,0500000US47111,0500000US47147,0500000US47161,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47119,0500000US21213,0500000US21003,1600000US2169114,1600000US2128918,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,0400000US47,0100000US
agebysex_total_series,204992,324139,187680,10910,232380,140604,40539,24208,70982,13553,690540,53289,8201,18528,94615,18319,21065,4489,8909,602,4720,2191,1220,42700,1711,271,2115,156092,39515,4528,1900,336,15500,1661,943,1706,8989,4868,80675,40262,16829,6859,57888,2744,5239,35556,34759,1802,6403,160,35834,4899,141704,667070,1930,9221,4536,2492,4660,12928,2441,123,50820,17147,41797,1296,6116,418,1698,4124,2622,3583,12258,6772268,326569308
age_total_male,102205,159299,91606,6668,113906,68930,20221,11894,35176,6572,332527,26246,4119,9113,45700,9099,10453,2059,4329,305,2298,1044,543,21154,8

In [70]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [71]:
data = transp

In [72]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [73]:
data.head()

NAME,"Montgomery County, Tennessee","Rutherford County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Cheatham County, Tennessee","Macon County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Scottsville city, Kentucky","Franklin city, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee",Tennessee,United States,GNRC,GNRC Region,MPO,Rutherford Incorporated,Rutherford Unincorporated,Wilson Incorporated,Wilson Unincorporated,Cheatham Incorporated,Cheatham Unincorporated,Dickson Incorporated,Dickson Unincorporated,Humphreys Incorporated,Humphreys Unincorporated,Montgomery Incorporated,Montgomery Unincorporated
GEO_ID,0500000US47125,0500000US47149,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US47021,0500000US47111,0500000US47147,0500000US47161,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47119,0500000US21213,0500000US21003,1600000US2169114,1600000US2128918,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,0400000US47,0100000US,,,,None,None,None,None,None,None,None,None,None,None,None,None
agebysex_total_series,204992.0,324139.0,187680.0,10910.0,232380.0,140604.0,40539.0,24208.0,70982.0,13553.0,690540.0,53289.0,8201.0,18528.0,94615.0,18319.0,21065.0,4489.0,8909.0,602.0,4720.0,2191.0,1220.0,42700.0,1711.0,271.0,211

In [74]:
ok = data.transpose().reset_index()

In [75]:
ok.head()

,NAME,GEO_ID,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,healthcoverage_total_healthcare_series,healthcoverage_total_mallhealthcare,healthcoverage_mu6_all,healthcoverage_mu6_w,healthcoverage_mu6_wo,healthcoverage_m6to18_all,healthcoverage_m6to18_w,healthcoverage_m6to18_wo,healthcoverage_m19to25_all,healthcoverage_m19to25_w,healthcoverage_m19to25_wo,healthcoverage_m26to34_all,healthcoverage_m26to34_w,healthcoverage_m26to34_wo,healthcoverage_m35to44_all,healthcoverage_m35to44_w,healthcoverage_m35to44_wo,healthcoverage_m45to54_all,healthcoverage_m45to54_w,healthcoverage_m45to54_wo,healthcoverage_m55to64_all,healthcoverage_m55to64_w,healthcoverage_m55to64_wo,healthcoverage_m65to74_all,healthcoverage_m65to74_w,healthcoverage_m65to74_wo,healthcoverage_m75+_all,healthcoverage_m75+_w,healthcoverage_m75+_wo,healthcoverage_total_fallhealthcare,healthcoverage_fu6_all,healthcoverage_fu6_w,healthcoverage_fu6_wo,healthcoverage_f6to18_all,healthcoverage_f6to18_w,healthcoverage_f6to18_wo,healthcoverage_f19to25_all,healthcoverage_f19to25_w,healthcoverage_f19to25_wo,healthcoverage_f26to34_all,healthcoverage_f26to34_w,healthcoverage_f26to34_wo,healthcoverage_f35to44_all,healthcoverage_f35to44_w,healthcoverage_f35to44_wo,healthcoverage_f45to54_all,healthcoverage_f45to54_w,healthcoverage_f45to54_wo,healthcoverage_f55to64_all,healthcoverage_f55to64_w,healthcoverage_f55to64_wo,healthcoverage_f65to74_all,healthcoverage_f65to74_w,healthcoverage_f65to74_wo,healthcoverage_f75+_all,healthcoverage_f75+_w,healthcoverage_f75+_wo,healthcoverage_total_privatehealthcare_series,healthcoverage_total_mprivate,healthcoverage_mu6_privateall,healthcoverage_mu6_wprivate,healthcoverage_mu6_woprivate,healthcoverage_m6to18_privateall,healthcoverage_m6to18_wprivate,healthcoverage_m6to18_woprivate,healthcoverage_m19to25_privateall,healthcoverage_m19to25_wprivate,healthcoverage_m19to25_woprivate,healthcoverage_m26to34_privateall,healthcoverage_m26to34_wprivate,healthcoverage_m26to34_woprivate,healthcoverage_m35to44_privateall,healthcoverage_m35to44_wprivate,healthcoverage_m35to44_woprivate,healthcoverage_m45to54_privateall,healthcoverage_m45to54_wprivate,healthcoverage_m45to54_woprivate,healthcoverage_m55to64_privateall,healthcoverage_m55to64_wprivate,healthcoverage_m55to64_woprivate,healthcoverage_m65to74_privateall,healthcoverage_m65to74_wprivate,healthcoverage_m65to74_woprivate,healthcoverage_m75+_privateall,healthcoverage_m75+_wprivate,healthcoverage_m75+_woprivate,healthcoverage_total_fprivate,healthcoverage_fu6_privateall,healthcoverage_fu6_wprivate,healthcoverage_fu6_woprivate,healthcoverage_f6to18_privateall,healthcoverage_f6to18_wprivate,healthcoverage_f6to18_woprivate,healthcoverage_f19to25_privateall,healthcoverage_f19to25_wprivate,healthcoverage_f19to25_woprivate,healthcoverage_f26to34_privateall,healthcoverage_f26to34_wprivate,healthcoverage_f26to34_woprivate,healthcoverage_f35to44_privateall,healthcoverage_f35to44_wprivate,healthcoverage_f35to44_woprivate,healthcoverage_f45to54_privateall,healthcoverage_f45to54_wprivate,healthcoverage_f45to54_woprivate,healthcoverage_f55to64_privateall,healthcoverage_f55t

In [76]:
ok.tail()

,NAME,GEO_ID,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,healthcoverage_total_healthcare_series,healthcoverage_total_mallhealthcare,healthcoverage_mu6_all,healthcoverage_mu6_w,healthcoverage_mu6_wo,healthcoverage_m6to18_all,healthcoverage_m6to18_w,healthcoverage_m6to18_wo,healthcoverage_m19to25_all,healthcoverage_m19to25_w,healthcoverage_m19to25_wo,healthcoverage_m26to34_all,healthcoverage_m26to34_w,healthcoverage_m26to34_wo,healthcoverage_m35to44_all,healthcoverage_m35to44_w,healthcoverage_m35to44_wo,healthcoverage_m45to54_all,healthcoverage_m45to54_w,healthcoverage_m45to54_wo,healthcoverage_m55to64_all,healthcoverage_m55to64_w,healthcoverage_m55to64_wo,healthcoverage_m65to74_all,healthcoverage_m65to74_w,healthcoverage_m65to74_wo,healthcoverage_m75+_all,healthcoverage_m75+_w,healthcoverage_m75+_wo,healthcoverage_total_fallhealthcare,healthcoverage_fu6_all,healthcoverage_fu6_w,healthcoverage_fu6_wo,healthcoverage_f6to18_all,healthcoverage_f6to18_w,healthcoverage_f6to18_wo,healthcoverage_f19to25_all,healthcoverage_f19to25_w,healthcoverage_f19to25_wo,healthcoverage_f26to34_all,healthcoverage_f26to34_w,healthcoverage_f26to34_wo,healthcoverage_f35to44_all,healthcoverage_f35to44_w,healthcoverage_f35to44_wo,healthcoverage_f45to54_all,healthcoverage_f45to54_w,healthcoverage_f45to54_wo,healthcoverage_f55to64_all,healthcoverage_f55to64_w,healthcoverage_f55to64_wo,healthcoverage_f65to74_all,healthcoverage_f65to74_w,healthcoverage_f65to74_wo,healthcoverage_f75+_all,healthcoverage_f75+_w,healthcoverage_f75+_wo,healthcoverage_total_privatehealthcare_series,healthcoverage_total_mprivate,healthcoverage_mu6_privateall,healthcoverage_mu6_wprivate,healthcoverage_mu6_woprivate,healthcoverage_m6to18_privateall,healthcoverage_m6to18_wprivate,healthcoverage_m6to18_woprivate,healthcoverage_m19to25_privateall,healthcoverage_m19to25_wprivate,healthcoverage_m19to25_woprivate,healthcoverage_m26to34_privateall,healthcoverage_m26to34_wprivate,healthcoverage_m26to34_woprivate,healthcoverage_m35to44_privateall,healthcoverage_m35to44_wprivate,healthcoverage_m35to44_woprivate,healthcoverage_m45to54_privateall,healthcoverage_m45to54_wprivate,healthcoverage_m45to54_woprivate,healthcoverage_m55to64_privateall,healthcoverage_m55to64_wprivate,healthcoverage_m55to64_woprivate,healthcoverage_m65to74_privateall,healthcoverage_m65to74_wprivate,healthcoverage_m65to74_woprivate,healthcoverage_m75+_privateall,healthcoverage_m75+_wprivate,healthcoverage_m75+_woprivate,healthcoverage_total_fprivate,healthcoverage_fu6_privateall,healthcoverage_fu6_wprivate,healthcoverage_fu6_woprivate,healthcoverage_f6to18_privateall,healthcoverage_f6to18_wprivate,healthcoverage_f6to18_woprivate,healthcoverage_f19to25_privateall,healthcoverage_f19to25_wprivate,healthcoverage_f19to25_woprivate,healthcoverage_f26to34_privateall,healthcoverage_f26to34_wprivate,healthcoverage_f26to34_woprivate,healthcoverage_f35to44_privateall,healthcoverage_f35to44_wprivate,healthcoverage_f35to44_woprivate,healthcoverage_f45to54_privateall,healthcoverage_f45to54_wprivate,healthcoverage_f45to54_woprivate,healthcoverage_f55to64_privateall,healthcoverage_f55t

In [77]:
ok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Columns: 1221 entries, NAME to GeoFIPS
dtypes: float64(1219), object(2)
memory usage: 858.6+ KB


In [79]:
ok.to_csv('../../data/ACS20205YR.csv', index = False)